# 🧬 Optimización de KNN con Algoritmo Genético para Aprobación de Préstamos
Este notebook implementa un algoritmo genético para optimizar los hiperparámetros de un modelo KNN, basado en un caso práctico del sector financiero: predicción de aprobación de préstamos.

## 📥 Carga del Dataset

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('dataset_finanzas.csv')
data.head()

### ✅ Conclusión: Carga de Datos
El dataset ha sido cargado correctamente. Contiene información relevante de clientes que servirá para predecir la aprobación de préstamos.

## ⚙️ Preparación del Dataset

In [ ]:
from sklearn.preprocessing import StandardScaler

X = data.drop('Aprobado', axis=1)
y = data['Aprobado']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

### ✅ Conclusión: Preparación de Datos
Los datos han sido divididos y escalados correctamente para el entrenamiento del modelo KNN.

## 🧪 Definición de Funciones del Algoritmo Genético

In [ ]:
import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

CHROMOSOME_LENGTH = 5
POP_SIZE = 10
GENERATIONS = 15
MUTATION_RATE = 0.1

def generate_chromosome():
    return ''.join(random.choice('01') for _ in range(CHROMOSOME_LENGTH))

def decode(chrom):
    k = int(chrom[0:4], 2) + 1
    metric = 'euclidean' if chrom[4] == '0' else 'manhattan'
    return k, metric

def fitness(chrom):
    k, metric = decode(chrom)
    knn = KNeighborsClassifier(n_neighbors=k, metric=metric)
    knn.fit(X_train, y_train)
    pred = knn.predict(X_test)
    return accuracy_score(y_test, pred)

def selection(pop):
    subset = random.sample(pop, 3)
    return max(subset, key=fitness)

def crossover(p1, p2):
    point = random.randint(1, CHROMOSOME_LENGTH - 1)
    return p1[:point] + p2[point:], p2[:point] + p1[point:]

def mutate(chrom):
    return ''.join(
        bit if random.random() > MUTATION_RATE else ('0' if bit == '1' else '1')
        for bit in chrom
    )

### ✅ Conclusión: Definición del Algoritmo Genético
Las funciones principales del algoritmo genético han sido definidas: generación, decodificación, evaluación (fitness), selección, cruce y mutación.

## 🔁 Ejecución del Algoritmo Genético

In [ ]:
population = [generate_chromosome() for _ in range(POP_SIZE)]
progress = []

for gen in range(GENERATIONS):
    population = sorted(population, key=fitness, reverse=True)
    best = population[0]
    progress.append(fitness(best))
    print(f"Gen {gen}: Accuracy = {fitness(best):.4f} | Parámetros: {decode(best)}")

    new_population = population[:2]
    while len(new_population) < POP_SIZE:
        p1 = selection(population)
        p2 = selection(population)
        c1, c2 = crossover(p1, p2)
        new_population.extend([mutate(c1), mutate(c2)])
    population = new_population[:POP_SIZE]

### ✅ Conclusión: Ejecución Evolutiva
Se han generado múltiples generaciones de soluciones usando AG, y se observa cómo mejora la precisión con cada iteración.

## 📈 Visualización del Progreso

In [ ]:
import matplotlib.pyplot as plt
plt.plot(progress, marker='o')
plt.title('Evolución del Accuracy - AG + KNN')
plt.xlabel('Generación')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

### ✅ Conclusión: Visualización del Resultado
El gráfico muestra cómo el algoritmo genético permitió encontrar configuraciones de KNN cada vez más precisas con el paso de las generaciones.

## ✅ Conclusión Final
El algoritmo genético demostró ser efectivo para optimizar hiperparámetros en el modelo KNN aplicado al problema de predicción de aprobación de préstamos. Este enfoque permite automatizar la selección de parámetros óptimos y mejora la eficiencia en tareas de modelado predictivo en entornos empresariales reales.